# processor.py

In [1]:
from asyncio.log import logger
import os, ast
from config import Config
from datetime import datetime, timezone
import logging
import boto3
from pyspark.sql import SparkSession
from path import *
from sub_processor import SubProcessor
from iceberg_catalog import IcebergCatalog
from broadcast import BroadCast
from udf import load_broadcast
import asyncio

import gc

# Config
AWS_PROFILE = Config.AWS_PROFILE

FILE_NAME = f"processor_execution_logs_{CURRENT_DATE}.log"
LOG_DIR = os.path.join("logs", CURRENT_DATE)
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, FILE_NAME)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler(LOG_FILE), logging.StreamHandler()],
)

class Processor:
    def __init__(self):

        self.broadcast = BroadCast()
        self.sub_processor = SubProcessor(self.broadcast)
        self.iceberg_catalog = IcebergCatalog()


        # Prepare Session
        if DEBUG:
            self.spark_session = (
                SparkSession.builder.appName("Details Data Process")
                .master("local[*]")
                .config("spark.executor.memory", "3g")
                #.config("spark.memory.fraction", "0.7")
                .config("spark.driver.memory", "4g")
                #.config("spark.executor.cores", "2")
                # .config("spark.driver.cores", "1")
                # .config("spark.executor.instances", "1")
                # .config("spark.memory.offHeap.enabled", "true")
                # .config("spark.memory.offHeap.size", "2g")
                .config(
                    "spark.jars.packages",
                    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2,"
                )
                .config(
                    "spark.sql.extensions",
                    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
                )
                .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
                .config("spark.sql.catalog.local.type", "hadoop")
                .config("spark.sql.catalog.local.warehouse", CURATED_ZONE)
                # .config("spark.executor.memoryOverhead", "1g")
                # .config("spark.driver.memoryOverhead", "1g")
                # .config("spark.driver.cores", "1")
                .getOrCreate()
            )

        else:
            # Get AWS Profile Credentials
            self.boto3_session = boto3.Session(profile_name=AWS_PROFILE)
            self.credentials = self.boto3_session.get_credentials()
            self.aws_access_key_id = self.credentials.access_key
            self.aws_secret_access_key = self.credentials.secret_key
            self.aws_session_token = self.credentials.token

            self.spark_session = (
                SparkSession.builder.appName("Details Data Process")
                .config(
                    "spark.jars.packages",
                    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2,"
                    "org.apache.hadoop:hadoop-aws:2.10.1,"
                    "com.amazonaws:aws-java-sdk-bundle:1.11.1026",
                )
                .config(
                    "spark.sql.extensions",
                    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
                )
                .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
                .config("spark.sql.catalog.local.type", "hadoop")
                .config("spark.sql.catalog.local.warehouse", CURATED_ZONE)
                .config(
                    "spark.sql.catalog.local.hadoop.fs.s3a.access.key",
                    self.aws_access_key_id,
                )
                .config(
                    "spark.sql.catalog.local.hadoop.fs.s3a.secret.key",
                    self.aws_secret_access_key,
                )
                .config(
                    "spark.sql.catalog.local.hadoop.fs.s3a.session.token",
                    self.aws_session_token,
                )
                .config(
                    "spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"
                )
                .config("spark.hadoop.fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")
                .config(
                    "spark.hadoop.fs.s3a.aws.credentials.provider",
                    "com.amazonaws.auth.DefaultAWSCredentialsProviderChain",
                )
                .config("spark.executor.memory", "7g")
                .config("spark.executor.memoryOverhead", "1g")
                .config("spark.driver.memory", "6g")
                .config("spark.driver.memoryOverhead", "1g")
                .config("spark.executor.cores", "3")
                .config("spark.driver.cores", "1")
                .config("spark.executor.instances", "8")
                .getOrCreate()
            )

    def preprocess_details_data(
            self,
            df_details,
            df_reviews_scores,
            df_chain_and_brand,
            df_search,
            ):
        df_processed = df_details

        logging.info("======== Basic data process started ========")
        df_processed = self.sub_processor.process_basic_data(df_processed=df_processed, spark_session=self.spark_session)
        logging.info("=======: df_processed : =====")
        return df_processed
    
    def preprocess_reviews_data(self,
            df_details,
            df_reviews_scores,
            df_chain_and_brand,
            df_search,
            ):
        df_processed = df_details
        logging.info("======== Before Review data process start ========")
        before_df_processed = self.sub_processor.process_review_scores_data(
            df_processed=df_processed, review_score_df=df_reviews_scores
        )
        logging.info("======== Before Review data process ended ========")

        logging.info("======== After Review data process start ========")
        after_df_processed = self.sub_processor.after_process_review_scores_data(
            df_processed=df_processed, review_score_df=df_reviews_scores
        )
        logging.info("======== After Review data process ended ========")

        return before_df_processed,after_df_processed
    def preprocess_property_flags_data(self,
            df_details,
            df_reviews_scores,
            df_chain_and_brand,
            df_search,):
        df_processed = df_details
        logging.info("======== Before Property Flags data process Started ========")
        befor_df_processed = self.sub_processor.process_property_flags_data(df_processed=df_processed)
        logging.info("======== Before Property Flags data process ended ========")

        logging.info("======== After Property Flags data process Started ========")
        after_df_processed = self.sub_processor.after_process_property_flags_data(df_processed=df_processed)
        logging.info("======== After Property Flags data process ended ========")

        return befor_df_processed,after_df_processed
    def preprocess_basic_dataprocess_chain_and_brand_data(self,
            df_details,
            df_reviews_scores,
            df_chain_and_brand,
            df_search,):
        df_processed = df_details
        logging.info("======== Before Chain and Brand data process Started ========")
        before_df_processed = self.sub_processor.process_chain_and_brand_data(
            df_processed=df_processed, df_chain_and_brand=df_chain_and_brand
        )
        logging.info("======== Before Chain and Brand data process Ended ========")
        
        logging.info("======== After Chain and Brand data process Started ========")
        after_df_processed = self.sub_processor.after_process_chain_and_brand_data(
            df_processed=df_processed, df_chain_and_brand=df_chain_and_brand
        )
        logging.info("======== After Chain and Brand data process Ended ========") 
        return before_df_processed,after_df_processed 
    
    def preprocess_commission_and_meal_plan_data(self,
            df_details,
            df_reviews_scores,
            df_chain_and_brand,
            df_search,):
        df_processed = df_details
        logging.info("======== Before Commission and meal plan data process Started ========")
        before_df_processed = self.sub_processor.process_commission_and_meal_plan_data(df_processed=df_processed, df_search=df_search)
        logging.info("======== After Commission and meal plan data process Ended ========")
        
        logging.info("======== After Commission and meal plan data process Started ========")
        after_df_processed = self.sub_processor.after_process_commission_and_meal_plan_data(df_processed=df_processed, df_search=df_search)
        logging.info("======== After Commission and meal plan data process Ended ========")
        return before_df_processed,after_df_processed 
        


    def process_details_data(
            self,
            df_details,
            df_reviews_scores,
            df_chain_and_brand,
            df_search,
    ):
        df_processed = df_details

        # Process Basic Data
        # logging.info("======== Basic data process started ========")
        # df_processed = self.sub_processor.process_basic_data(df_processed=df_processed, spark_session=self.spark_session)
        # logging.info("=======: df_processed : =====")
        # df_processed.show(5)
        logging.info("======== Basic data process ended ========")

        logging.info("======== Basic data process Started ========")
        df_processed = self.sub_processor.process_review_scores_data(
            df_processed=df_processed, review_score_df=df_reviews_scores
        )
        logging.info("======== Review data process ended ========")

        logging.info("======== Property Flags data process Started ========")
        df_processed = self.sub_processor.process_property_flags_data(df_processed=df_processed)
        logging.info("======== Property Flags data process ended ========")

        logging.info("======== Chain and Brand data process Started ========")
        df_processed = self.sub_processor.process_chain_and_brand_data(
            df_processed=df_processed, df_chain_and_brand=df_chain_and_brand
        )
        logging.info("======== Chain and Brand data process Ended ========")

        # df_processed = self.sub_processor.process_property_flags_data(df_processed=df_processed)

        logging.info("======== Commission and meal plan data process Started ========")
        df_processed = self.sub_processor.process_commission_and_meal_plan_data(df_processed=df_processed, df_search=df_search)
        logging.info("======== Commission and meal plan data process Ended ========")

        logging.info("======== USD price and price history data process Started ========")
        df_processed, df_price_history = self.sub_processor.process_usd_price_and_price_history(df_processed=df_processed, df_search=df_search, spark_session=self.spark_session)
        logging.info("======== USD price and price history data process Ended ========")

        return df_processed, df_price_history

    def process_details_localize_data(self, df_processed):
        df_processed = self.sub_processor.process_localize_data(df_processed=df_processed, spark_session=self.spark_session)
        return df_processed
    
    def read_df_details(self):
        # # Load Details Data
        logging.info("Loading Details Data =======")
        df_details = self.spark_session.read.format("json").option("multiline", "true").load(DETAILS_DATA_DIR)
        logging.info("Details Data Loaded =======")
        return df_details
    
    def read_df_reviews_scores(self):
        logging.info("Loading Reviews Scores Data =======")
        df_reviews_scores = self.spark_session.read.format("json").option("multiline", "true").load(REVIEW_SCORES_DIR)
        logging.info("Reviews Scores Data Loaded =======")
        return df_reviews_scores

                
    def read_df_search(self):
        logging.info("Loading Reviews Scores Data =======")
        df_search = self.spark_session.read.format("json").option("multiline", "true").load(ACCOMMODATION_SEARCH_DIR)
        logging.info("Search Data Loaded =======")
        return df_search
    
    def read_df_chain_and_brand(self):
        logging.info("Chain and Brand Data Loaded =======")
        df_chain_and_brand = self.spark_session.read.format("json").option("multiline", "true").load(CHAIN_AND_BRAND)
        return df_chain_and_brand
    
    def read_df_reviews(self):
        logging.info("Reviews Data Loaded =======")
        df_reviews = self.spark_session.read.format("json").option("multiline", "true").load(REVIEW_DIR)
        return df_reviews
    
    def read_df_processed(self,df_details):
        logging.info("Processed Data Loaded =======")
        df_processed, count_df_null_location_id = self.sub_processor.process_location_data_bg(df_details, self.spark_session)
        return df_processed,count_df_null_location_id
    
    def BroadCast(self):
        # Load Broadcast Data
        logging.info("====== Broadcasting Data =======")
        self.broadcast.prepare_broadcasted_data(spark_session=self.spark_session)
        load_broadcast(self.broadcast)
        logging.info("====== Broadcasted Data =======") 

    def preprocess_basic_data(self,df_processed,df_reviews_scores,df_chain_and_brand,df_search):
        logging.info("====== Details Data process started =======")
        df_processed = self.preprocess_details_data(
            df_details=df_processed,
            df_reviews_scores=df_reviews_scores,
            df_chain_and_brand=df_chain_and_brand,
            df_search=df_search
        )
        return df_processed
    



In [2]:
processor = Processor()

25/02/18 17:36:47 WARN Utils: Your hostname, w3e103 resolves to a loopback address: 127.0.1.1; using 192.168.0.243 instead (on interface enp2s0)
25/02/18 17:36:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/w3e103/Downloads/project/inventroy-cron/env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/w3e103/.ivy2/cache
The jars for the packages stored in: /home/w3e103/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-274383ad-458e-4c13-b045-ff338fc85f5c;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
:: resolution report :: resolve 286ms :: artifacts dl 6ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.sp

# read details data

In [3]:
df_details=processor.read_df_details()

2025-02-18 17:36:52,944 - INFO - Loading Details Data =======
2025-02-18 17:39:12,247 - INFO - Details Data Loaded =======                    


In [4]:
df_details.count()

530865

# read reviews score data

In [5]:
df_reviews_scores=processor.read_df_reviews_scores()

2025-02-18 17:40:35,790 - INFO - Loading Reviews Scores Data =======
2025-02-18 17:41:53,207 - INFO - Reviews Scores Data Loaded =======             


In [6]:
df_reviews_scores.count()

410729

# read search data

In [ ]:
df_search=processor.read_df_search()

2025-02-18 17:41:58,390 - INFO - Loading Reviews Scores Data =======


In [ ]:
df_search.count()

# read chain and brand data

In [ ]:
df_chain_and_brand=processor.read_df_chain_and_brand()

2025-02-18 16:50:32,354 - INFO - Chain and Brand Data Loaded =======


In [ ]:
df_chain_and_brand.count()

# read reviews data

In [ ]:
df_reviews=processor.read_df_reviews()

2025-02-18 16:50:32,880 - INFO - Reviews Data Loaded =======


In [ ]:
df_reviews.count()

# read processed data

In [ ]:
df_processed,count_df_null_location_id = processor.read_df_processed(df_details)

2025-02-18 16:50:47,103 - INFO - Processed Data Loaded =======


In [ ]:
df_processed.count()

# delete duplicate data

In [ ]:
df_details = df_details.dropDuplicates(["id"]).limit(100)
df_reviews_scores = df_reviews_scores.dropDuplicates(["id"]).limit(100)
df_search = df_search.dropDuplicates(["id"]).limit(100)
df_reviews = df_reviews.dropDuplicates(["id"]).limit(100)
logging.info("!!!Data count !!!")
print(df_details.count())
logging.info("====== Duplicates Removed =======")

2025-02-18 16:51:16,514 - INFO - !!!Data count !!!


2025-02-18 16:51:22,285 - INFO - ====== Duplicates Removed =======              


100


# load broadcast

In [ ]:
processor.BroadCast()

2025-02-18 16:51:22,360 - INFO - ====== Broadcasting Data =======
2025-02-18 16:51:22,605 - INFO - ====== Broadcasted Data =======


# preprocess basic data

In [ ]:
df_processed = processor.preprocess_basic_data(df_details, df_reviews_scores, df_chain_and_brand, df_search)

2025-02-18 16:51:22,723 - INFO - ====== Details Data process started =======
2025-02-18 16:51:22,732 - INFO - ======== Basic data process started ========
2025-02-18 16:51:22,793 - INFO - >> sub-process : property_name() completed =====
2025-02-18 16:51:23,305 - INFO - >> sub-process : process_rating_and_score() completed =====
2025-02-18 16:51:23,537 - INFO - >> sub-process : process_checkin_checkout_times() completed =====
2025-02-18 16:51:23,820 - INFO - >> sub-process : process_property_slug() completed =====
2025-02-18 16:51:24,134 - INFO - >> sub-process : process_property_type() completed =====
2025-02-18 16:51:24,328 - INFO - >> sub-process : process_property_type_category() completed =====
2025-02-18 16:51:24,527 - INFO - >> sub-process : process_property_type_categories() completed =====
2025-02-18 16:51:24,617 - INFO - >> sub-process : process_room_type() completed =====
2025-02-18 16:51:28,892 - INFO - >> sub-process : process_amenities_data() completed =====
2025-02-18 16:

In [ ]:
#df_processed.show(5)

# preprocess review scores data

In [ ]:
before_df_reviews,after_df_reviews = processor.preprocess_reviews_data(df_processed,df_reviews_scores, df_chain_and_brand, df_search)

2025-02-18 16:51:36,689 - INFO - ======== Before Review data process start ========


2025-02-18 16:51:38,460 - INFO - ======== Before Review data process ended ========
2025-02-18 16:51:38,466 - INFO - ======== After Review data process start ========
2025-02-18 16:51:39,572 - INFO - ==== : After process reviews data : ======
2025-02-18 16:51:39,574 - INFO - ======== After Review data process ended ========


In [ ]:
#before_df_reviews.show(5)

25/02/18 16:51:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/02/18 16:52:03 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:52:48 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
# after_df_reviews.show(5)

25/02/18 16:53:18 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:53:47 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
df_processed=before_df_reviews

# process property flag data

In [ ]:
before_df_property_flag,after_df_property_flag = processor.preprocess_property_flags_data(df_processed,df_reviews_scores, df_chain_and_brand, df_search)

2025-02-18 16:53:50,194 - INFO - ======== Before Property Flags data process Started ========
2025-02-18 16:53:55,525 - INFO - ======== Before Property Flags data process ended ========
2025-02-18 16:53:55,746 - INFO - ======== After Property Flags data process Started ========
2025-02-18 16:53:59,644 - INFO - ===== : Optimized process Property Flags Data Complete : =====
2025-02-18 16:53:59,645 - INFO - ======== After Property Flags data process ended ========


In [ ]:
#before_df_property_flag.show(5)

25/02/18 16:54:19 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:54:42 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
#after_df_property_flag.show(5)

25/02/18 16:55:00 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:55:21 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
df_processed=before_df_property_flag

# process chain and brand data

In [ ]:
before_df_chian_brand,after_df_chian_brand = processor.preprocess_basic_dataprocess_chain_and_brand_data(df_processed,df_reviews_scores, df_chain_and_brand, df_search)

2025-02-18 16:55:22,632 - INFO - ======== Before Chain and Brand data process Started ========


2025-02-18 16:55:23,589 - INFO - ======== Before Chain and Brand data process Ended ========
2025-02-18 16:55:23,607 - INFO - ======== After Chain and Brand data process Started ========
2025-02-18 16:55:24,074 - INFO - === : After Process Chain and Brand Data :====
2025-02-18 16:55:24,076 - INFO - ======== After Chain and Brand data process Ended ========


In [ ]:
before_df_chian_brand.show(5)

25/02/18 16:55:40 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:56:00 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
after_df_chian_brand.show(5)

25/02/18 16:56:19 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:56:41 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
df_processed=before_df_chian_brand

# process commission and meal plan data

In [ ]:
before_df_commission_meal,after_df_commission_meal = processor.preprocess_commission_and_meal_plan_data(df_processed,df_reviews_scores, df_chain_and_brand, df_search)

2025-02-18 16:56:42,831 - INFO - ======== Before Commission and meal plan data process Started ========


2025-02-18 16:56:43,749 - INFO - ======== After Commission and meal plan data process Ended ========
2025-02-18 16:56:43,775 - INFO - ======== After Commission and meal plan data process Started ========
2025-02-18 16:56:44,178 - INFO - ===== : Optimized process Commission and Meal Plan Data Complete : =====
2025-02-18 16:56:44,179 - INFO - ======== After Commission and meal plan data process Ended ========


In [ ]:
# before_df_commission_meal.show(5)

25/02/18 16:57:00 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:57:24 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
# after_df_commission_meal.show(5)

25/02/18 16:57:43 WARN DAGScheduler: Broadcasting large task binary with size 1961.3 KiB
25/02/18 16:58:05 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+--------+------------------+------+----------------------+--------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------+------------+--------------------+------------+------------+----------+-------------+-----------+--------------------+----------+-------------+----------------------+------------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------------+---------------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+--------------+-------------+--------------+--------+---------+--------------------+----------------+---

In [ ]:
df_processed=before_df_commission_meal

# 